<a href="https://colab.research.google.com/github/knickka/Recommender-System/blob/main/Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import collections
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movie_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Recommender System/tmdb_movies_data.csv')

In [3]:
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10866 non-null  int64  
 1   imdb_id               10856 non-null  object 
 2   popularity            10866 non-null  float64
 3   budget                10866 non-null  int64  
 4   revenue               10866 non-null  int64  
 5   original_title        10866 non-null  object 
 6   cast                  10790 non-null  object 
 7   homepage              2936 non-null   object 
 8   director              10822 non-null  object 
 9   tagline               8042 non-null   object 
 10  keywords              9373 non-null   object 
 11  overview              10862 non-null  object 
 12  runtime               10866 non-null  int64  
 13  genres                10843 non-null  object 
 14  production_companies  9836 non-null   object 
 15  release_date       

In [4]:
movie_data.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,137999939.3,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,137999939.3,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,101199955.5,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,183999919.0,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015,174799923.1,1.385749e+09


In [5]:
movie_data = movie_data[['id','original_title','cast','director','overview','genres']]

In [6]:
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10866 entries, 0 to 10865
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              10866 non-null  int64 
 1   original_title  10866 non-null  object
 2   cast            10790 non-null  object
 3   director        10822 non-null  object
 4   overview        10862 non-null  object
 5   genres          10843 non-null  object
dtypes: int64(1), object(5)
memory usage: 509.5+ KB


In [7]:
movie_data.head()

,id,original_title,cast,director,overview,genres
0,135397,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,Twenty-two years after the events of Jurassic ...,Action|Adventure|Science Fiction|Thriller
1,76341,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,An apocalyptic story set in the furthest reach...,Action|Adventure|Science Fiction|Thriller
2,262500,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,Beatrice Prior must confront her inner demons ...,Adventure|Science Fiction|Thriller
3,140607,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Thirty years after defeating the Galactic Empi...,Action|Adventure|Science Fiction|Fantasy
4,168259,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Deckard Shaw seeks revenge against Dominic Tor...,Action|Crime|Thriller


In [8]:
movie_data['cast'][0]

"Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vincent D'Onofrio|Nick Robinson"

In [9]:

movie_data['original_title']=movie_data['original_title'].apply(lambda x: str(x).replace(" ",""))
movie_data['original_title']=movie_data['original_title'].apply(lambda x: str(x).split())

movie_data['cast']=movie_data['cast'].apply(lambda x: str(x).replace(" ",""))
movie_data['cast']=movie_data['cast'].apply(lambda x: str(x).replace("|"," "))
movie_data['cast']=movie_data['cast'].apply(lambda x: str(x).split())

movie_data['director']=movie_data['director'].apply(lambda x: str(x).replace(" ",""))
movie_data['director']=movie_data['director'].apply(lambda x: str(x).split())

movie_data['overview']=movie_data['overview'].apply(lambda x: str(x).split())

movie_data['genres']=movie_data['genres'].apply(lambda x: str(x).replace(" ",""))
movie_data['genres']=movie_data['genres'].apply(lambda x: str(x).replace("|"," "))
movie_data['genres']=movie_data['genres'].apply(lambda x: str(x).split())

In [10]:
def first_three(list):
  new_list = []
  if len(list) > 3:
    for i in range(3):
      new_list.append(list[i])

  return new_list

In [11]:
movie_data['cast']=movie_data['cast'].apply(first_three)


In [12]:
movie_data.head()

,id,original_title,cast,director,overview,genres
0,135397,[JurassicWorld],"[ChrisPratt, BryceDallasHoward, IrrfanKhan]",[ColinTrevorrow],"[Twenty-two, years, after, the, events, of, Ju...","[Action, Adventure, ScienceFiction, Thriller]"
1,76341,[MadMax:FuryRoad],"[TomHardy, CharlizeTheron, HughKeays-Byrne]",[GeorgeMiller],"[An, apocalyptic, story, set, in, the, furthes...","[Action, Adventure, ScienceFiction, Thriller]"
2,262500,[Insurgent],"[ShaileneWoodley, TheoJames, KateWinslet]",[RobertSchwentke],"[Beatrice, Prior, must, confront, her, inner, ...","[Adventure, ScienceFiction, Thriller]"
3,140607,[StarWars:TheForceAwakens],"[HarrisonFord, MarkHamill, CarrieFisher]",[J.J.Abrams],"[Thirty, years, after, defeating, the, Galacti...","[Action, Adventure, ScienceFiction, Fantasy]"
4,168259,[Furious7],"[VinDiesel, PaulWalker, JasonStatham]",[JamesWan],"[Deckard, Shaw, seeks, revenge, against, Domin...","[Action, Crime, Thriller]"


In [13]:
movie_data['tags'] = movie_data['original_title'] + movie_data['genres'] + movie_data['overview'] + movie_data['director'] + movie_data['cast']

In [14]:
movie_data['cast']

0              [ChrisPratt, BryceDallasHoward, IrrfanKhan]
1              [TomHardy, CharlizeTheron, HughKeays-Byrne]
2                [ShaileneWoodley, TheoJames, KateWinslet]
3                 [HarrisonFord, MarkHamill, CarrieFisher]
4                    [VinDiesel, PaulWalker, JasonStatham]
                               ...                        
10861    [MichaelHynson, RobertAugust, Lord'TallyHo'Ble...
10862            [JamesGarner, EvaMarieSaint, YvesMontand]
10863    [InnokentiySmoktunovskiy, OlegEfremov, GeorgiZ...
10864          [TatsuyaMihashi, AkikoWakabayashi, MieHama]
10865            [HaroldP.Warren, TomNeyman, JohnReynolds]
Name: cast, Length: 10866, dtype: object

In [15]:
data = movie_data[['id','original_title','tags']]

In [16]:
data

,id,original_title,tags
0,135397,[JurassicWorld],"[JurassicWorld, Action, Adventure, ScienceFict..."
1,76341,[MadMax:FuryRoad],"[MadMax:FuryRoad, Action, Adventure, ScienceFi..."
2,262500,[Insurgent],"[Insurgent, Adventure, ScienceFiction, Thrille..."
3,140607,[StarWars:TheForceAwakens],"[StarWars:TheForceAwakens, Action, Adventure, ..."
4,168259,[Furious7],"[Furious7, Action, Crime, Thriller, Deckard, S..."
...,...,...,...
10861,21,[TheEndlessSummer],"[TheEndlessSummer, Documentary, The, Endless, ..."
10862,20379,[GrandPrix],"[GrandPrix, Action, Adventure, Drama, Grand, P..."
10863,39768,[BeregisAvtomobilya],"[BeregisAvtomobilya, Mystery, Comedy, An, insu..."
10864,21449,"[What'sUp,TigerLily?]","[What'sUp,TigerLily?, Action, Comedy, In, comi..."


In [17]:
data['tags'] = data['tags'].apply(lambda x: " ".join(x))
data['tags'] = data['tags'].apply(lambda x: x.lower())

<ipython-input-17-0b08fbd00245>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tags'] = data['tags'].apply(lambda x: " ".join(x))
<ipython-input-17-0b08fbd00245>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tags'] = data['tags'].apply(lambda x: x.lower())


In [18]:
data

,id,original_title,tags
0,135397,[JurassicWorld],jurassicworld action adventure sciencefiction ...
1,76341,[MadMax:FuryRoad],madmax:furyroad action adventure sciencefictio...
2,262500,[Insurgent],insurgent adventure sciencefiction thriller be...
3,140607,[StarWars:TheForceAwakens],starwars:theforceawakens action adventure scie...
4,168259,[Furious7],furious7 action crime thriller deckard shaw se...
...,...,...,...
10861,21,[TheEndlessSummer],theendlesssummer documentary the endless summe...
10862,20379,[GrandPrix],grandprix action adventure drama grand prix dr...
10863,39768,[BeregisAvtomobilya],beregisavtomobilya mystery comedy an insurance...
10864,21449,"[What'sUp,TigerLily?]","what'sup,tigerlily? action comedy in comic woo..."


In [19]:
data['tags'][0]

'jurassicworld action adventure sciencefiction thriller twenty-two years after the events of jurassic park, isla nublar now features a fully functioning dinosaur theme park, jurassic world, as originally envisioned by john hammond. colintrevorrow chrispratt brycedallashoward irrfankhan'

In [20]:
len(data['tags'])

10866

In [21]:
tags = data['tags'].apply(lambda x: x.split())

In [22]:
tags

0        [jurassicworld, action, adventure, sciencefict...
1        [madmax:furyroad, action, adventure, sciencefi...
2        [insurgent, adventure, sciencefiction, thrille...
3        [starwars:theforceawakens, action, adventure, ...
4        [furious7, action, crime, thriller, deckard, s...
                               ...                        
10861    [theendlesssummer, documentary, the, endless, ...
10862    [grandprix, action, adventure, drama, grand, p...
10863    [beregisavtomobilya, mystery, comedy, an, insu...
10864    [what'sup,tigerlily?, action, comedy, in, comi...
10865    [manos:thehandsoffate, horror, a, family, gets...
Name: tags, Length: 10866, dtype: object

In [23]:
def build_vocab(data,vocab_size):
  vocab = []
  size = len(data)
  for i in range(size):
    for words in tags[i]:
      vocab.append(words)

  stopwords = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz",]
  vocab = [i for i in vocab if i not in stopwords]
  count = collections.Counter(vocab)
  vocab = sorted(count.items(),key=lambda x: (-x[1],x[0]))
  vocab = vocab[:vocab_size]
  words,count = list(zip(*vocab))
  return words, count




In [24]:
vocab,_ = build_vocab(tags,vocab_size=10000)

In [26]:
# vectorizer
# make a list of 10000 item with int(0) for each movie named vector
# if a word from a movie tags is in vocab, find the index of the word in vocab and +1 that index in vector
# do it for all the movies

In [27]:
def vectorizer(data,vocab,shape):
  matrix = np.zeros(shape)
  for i,words in enumerate(data):
    for word in words:
      if word in vocab:
        matrix[i][vocab.index(word)] += 1
  return matrix

In [28]:
shape = (len(tags),len(vocab))
vectorize = vectorizer(tags,vocab,shape)

In [29]:
vectors = cosine_similarity(vectorize)

In [30]:
vectors

array([[1.        , 0.17342199, 0.16366342, ..., 0.        , 0.09100315,
        0.        ],
       [0.17342199, 1.        , 0.09933993, ..., 0.        , 0.16571045,
        0.        ],
       [0.16366342, 0.09933993, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.04445542,
        0.        ],
       [0.09100315, 0.16571045, 0.        , ..., 0.04445542, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [39]:
data['original_title'] = data['original_title'].apply(lambda x: "".join(x))

<ipython-input-39-64bae1eb108a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['original_title'] = data['original_title'].apply(lambda x: "".join(x))


In [41]:
data['original_title'] = data['original_title'].apply(lambda x: x.lower())

<ipython-input-41-4460ad9797d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['original_title'] = data['original_title'].apply(lambda x: x.lower())


In [71]:
def get_sim_movies(name):
  ind = data[data['original_title'] == name].index[0]
  recom_ind = sorted(list(enumerate(vectors[ind])),key=lambda x:-x[1])[1:4]
  req_list = []
  for i in recom_ind:
    req_list.append(data['original_title'][i[0]])
  return req_list


In [78]:
get_sim_movies('kungfupanda')

['kungfupanda2', 'thesecretofnimh', 'kungfupanda:secretsofthefuriousfive']